# Big script for all prepocessing of the genome data

### Parcing genomes to extract genes of interest and translate them to proteins

#### Reading all the needed genes

In [2]:
import pandas as pd
import numpy as np
#load library of all annotated genes from Kira's paper
with open('./cas1903.isl.type.txt') as file:
    cas_voc = pd.read_csv(file, delimiter="\t", comment='=')
cas_voc['Position'].shape

(66309,)

In [3]:
# Add more convitient start and end fileds
cas_voc[['Start', 'End']] = cas_voc['Position'].astype(str).apply(lambda x: x.split('..')).tolist()
cas_voc.drop('Position', axis=1, inplace=True)

In [4]:
# Drop CRISPR arrays annotation
cas_voc["Protein_id"].str.replace("_1", ".1")
cas_voc = cas_voc.dropna(subset=['Protein_id'])
cas_voc.head()

,Loci_id,Strand,Genome_assembly,Chromosome,N1,Protein_id,B2,Gene_id,Gene_family,Type,Species,Start,End
0,TTMY_RS00505,-,GCF_002355995.1,NZ_AP017920.1,72.0,WP_096410491_1,+,cd09722,cas1,CAS-I-B,Thermus_thermophilus_TMY,100411,101394
1,TTMY_RS00510,-,GCF_002355995.1,NZ_AP017920.1,73.0,WP_096410492_1,+,pfam01930,cas4,CAS-I-B,Thermus_thermophilus_TMY,101387,101908
2,TTMY_RS00515,-,GCF_002355995.1,NZ_AP017920.1,74.0,WP_096410493_1,+,COG1203,cas3,CAS-I-B,Thermus_thermophilus_TMY,101884,103377
3,TTMY_RS00520,-,GCF_002355995.1,NZ_AP017920.1,75.0,WP_096410494_1,+,cd09641,cas3HD,CAS-I-B,Thermus_thermophilus_TMY,103385,104128
4,TTMY_RS00525,+,GCF_002355995.1,NZ_AP017920.1,76.0,WP_096412881_1,+,cd09652,cas6,CAS-I-B,Thermus_thermophilus_TMY,104172,104900


In [5]:
# Read genes_to_use list of approved for training gene-tags
genes_to_use = set()
with open("genes_to_use.tsv", "r") as file:
    for line in file:
        genes_to_use.add(line.split()[0])
genes_to_use.remove("Gene")

In [6]:
# Mask genes_to_use on cas_voc
cas_voc_1 = cas_voc[cas_voc["Gene_family"].apply(lambda x: x in genes_to_use)]
cas_voc = cas_voc_1
len(cas_voc)


37307

All needed genes and information about them is now in cas_voc

In [7]:
# Extract all needed genome assessions (typos included) for future download

assessions = cas_voc["Genome_assembly"].to_list()

In [8]:
filename = "./assessions.txt"   
ass = set(assessions)
with open(filename, "w") as f:
    for i in ass:
        f.write(i+"\n")



In [9]:
len(ass)

4838

Run commands from script.sh file

Unzipping is broken, you need to concatenate each "./ncbi_dataset/ncbi_dataset/data/dataset_catalog.json" file from archives into one big in the unzipped folder manually

### Reading all the assembly files, extracting sequences


In [10]:
# Read all genome files and assembly ids

import json
file_structure = pd.DataFrame(columns=["Assembly", "Path"])
filename = "./ncbi_dataset/ncbi_dataset/data/dataset_catalog.json"

# load the JSON data from a file or a string
with open(filename, 'r') as f:
    data = json.load(f)
    

# access the values using the keys
api_version = data['apiVersion']
assemblies = data['assemblies']

# iterate over the assemblies and access their values
for assembly in assemblies:
    accession = assembly.get('accession', 'N/A')
    files = assembly['files']
    for file in files:
        if (file['fileType'] == "GENOMIC_NUCLEOTIDE_FASTA"):
            df = pd.DataFrame({"Assembly":[str(accession)], "Path":[file["filePath"]]})
            file_structure = pd.concat([file_structure, df], ignore_index=True)


FileNotFoundError: [Errno 2] No such file or directory: './ncbi_dataset/ncbi_dataset/data/dataset_catalog.json'

Unfortunately not all genome accessions found ind ncbi's database

In [10]:
missing = pd.DataFrame(ass - set(file_structure["Assembly"]))
print(missing.head())
print(len(missing))

                 0
0  GCF_000147855.2
1  GCF_000021385.1
2  GCF_000014905.1
3  GCF_000018005.1
4  GCF_000013105.1
33


In [11]:
# Read all the genome assession files, check in cas_voc what genes are needed from that file, read those genes, add new filed for Seq into all needed genes from cas_voc 

# Takes ~5m

from Bio import SeqIO

# Reads assession files one by one and takes from them what's needed
def extract_from_fasta(sub_voc, current_assembly):
    filepath = "./ncbi_dataset/ncbi_dataset/data/"+current_assembly["Path"]
    new_sub_voc = pd.DataFrame()
    with open(filepath, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            for index, gene in sub_voc.iterrows():
                if (record.id == gene["Chromosome"]):
                    seq = record.seq[int(gene["Start"])-1 : int(gene["End"])]
                    seq = pd.Series({"Seq":(str(seq))})
                    gene_1 = pd.concat([gene, seq], axis=0)
                    new_sub_voc = pd.concat([new_sub_voc, gene_1.to_frame().T], axis = 0)
                    gene_1 = []
    if len(new_sub_voc) != len(sub_voc): #check that we found sequences for all genes requested in sub_voc
        print(new_sub_voc, sub_voc)
    return new_sub_voc


new_cas_voc = pd.DataFrame() # needed to not overwrite cas_voc in this cell
for index, current_assembly in file_structure.iterrows():
    sub_voc = cas_voc[cas_voc["Genome_assembly"] == current_assembly["Assembly"]] # subsection of cas_voc that is referring to this genome assembly file
    new_cas_voc = pd.concat([new_cas_voc, extract_from_fasta(sub_voc, current_assembly)])



In [17]:
s1 = set(file_structure["Assembly"])
s2 = set(cas_voc["Genome_assembly"])
diff = s2 - s1
len(diff) == len(missing)

# We only lost genomes that weren't found on ncbi during download phase

False

In [18]:
cas_voc = new_cas_voc # once we are confident that new_cas_voc constructed normally we upadete 
cas_voc

,Loci_id,Strand,Genome_assembly,Chromosome,N1,Protein_id,B2,Gene_id,Gene_family,Type,Species,Start,End,Seq,Prot
0,RN99_05230,-,GCA_001296125.1,CP012714.1,991.0,ALF19893_1,+,cd09644,csn2,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1100111,1100773,TTAGAAAATTTCACATAAATCATTATCTATAACAATTAAATTATCA...,MTFQYKGFNFKIDFEEKNIFSLIVENKRAYRKIIEDLVNNSNIEDG...
0,RN99_05235,-,GCA_001296125.1,CP012714.1,992.0,ALF20727_1,+,mkCas0206,cas2,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1100770,1101075,TCATAAAACCACAAGCCTTTCATCTGTTTCTAAAAATGTCCCTTTT...,MRMLLFFDLPSVTNSDLKEYRKFRKFLIENGFSMLQESVYSKLLLH...
0,RN99_05240,-,GCA_001296125.1,CP012714.1,993.0,ALF19894_1,+,cd09720,cas1,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1101080,1101958,CTATAACTCATCTTGAAAAAATCTCACTAATGATAAATCATTTGAG...,MSGWRVIIVTGRGKLDLRYNSISIRRDNGTDFIHIGEVNTLILETT...
0,RN99_05245,-,GCA_001296125.1,CP012714.1,994.0,ALF19895_1,+,mkCas0193,cas9,CAS-II-A,Fusobacterium_nucleatum_sub_vincentii_ChDC_F8_...,1101985,1106109,TTATAGTTTAATTTTCTTTACAAAAAGCCCTGTAACTGATTCTTCT...,MKKQKFSDYYLGFDIGTNSVGWCVTDLDYNVLRFNKKDMWGSRLFD...
0,Tel_12180,-,GCA_001447805.1,CP013099.1,2369.0,ALP53829_1,+,pfam09618,cas6f,CAS-I-F,Candidatus_Tenderia_electrophaga,2660425,2660991,TCAAAACCAAGGAATGGTGGCTTCGTTACTCAGACCGTAGGTGTTG...,MNRYQNIKILPDPEFPAPMLINALFAKLHRALVALQSREIGVSFPK...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,DQN35_RS05450,-,GCF_900476015.1,NZ_LS483442.1,987.0,WP_011285506_1,+,COG3513,cas9,CAS-II-A,Streptococcus_pyogenes_NCTC8370,1028203,1032309,TCAGTCACCTCCTAGCTGACTCAAATCAATGCGTGTTTCATAAAGA...,MDKKYSIGLDIGTNSVGWAVITDDYKVPSKKFKVLGNTDRHSIKKN...
0,EL095_RS09000,-,GCF_900636345.1,NZ_LR134264.1,1709.0,WP_103364182_1,+,mkCas0216,csn2,CAS-II-A,Staphylococcus_hyicus_NCTC7944,1872256,1873107,CTAATATTTGTTCATAAATTGAATGAAGAACTCATCATCAAGATAT...,MKLVGEFNKPIEIAKQRLNVISFQNTSKLNALTEGLIKYSQSKSKN...
0,EL095_RS09005,-,GCF_900636345.1,NZ_LR134264.1,1710.0,WP_051035996_1,+,mkCas0206,cas2,CAS-II-A,Staphylococcus_hyicus_NCTC7944,1873104,1873412,TCATAATTCAAGAATTCTTTCTGTTGTATTTACATCATATTTCGTT...,LRLFIMFDLPVETSRERREYRKFVKFLLNEGYVRSQYSIYCKLILN...
0,EL095_RS09010,-,GCF_900636345.1,NZ_LR134264.1,1711.0,WP_103364183_1,+,cd09720,cas1,CAS-II-A,Staphylococcus_hyicus_NCTC7944,1873447,1874322,TTAGAAGTTAAAAACAGGCATCTTCATAAAGTTGATATCTCCACTT...,MSFRTVIITKESKLSLRMNHLIVKADELYKIPLQEILCLVIENPSV...


In [16]:
output = "./cas_dataset_kira.tsv"
cas_voc = pd.read_csv(output, sep='\t')

'MTFQYKGFNFKIDFEEKNIFSLIVENKRAYRKIIEDLVNNSNIEDGDIILSKNNKLVIPEKEIFIFSDYFNFDINKFVLNKYYKELKNLSENEFLNETLEIKEILKDYINKLIENNYSLKFEDDLDVSQILKAFSIKFERSEDLLLNLFEWLKILNEILGYEIFFFINIENFLSEDELLEFSKFIVYNKYKVVFLENFYRNKLSDNDNLIVIDNDLCEIF*'

In [36]:
# Protein construction

from Bio.Seq import Seq

def prot(gene):
    sequence = Seq(str(gene["Seq"]))
    
    if gene["Strand"] == "+":
        return str(sequence.translate(stop_symbol=" "))
    else:
        return str(sequence.reverse_complement().translate(stop_symbol=" "))
    
cas_voc["Prot"] = cas_voc.apply(prot, axis=1)
cas_voc["Prot"][1547]

c:\Users\Dart_ilder\miniconda3\envs\.conda_Cas\lib\site-packages\Bio\Seq.py:2979: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


'VYVAEWRLVELRLRAEGPCPMAGWSGSAAYRILLETTRREAEPKNKIYAHPIYVGGRPLLTGVDGRAAVLEPGTRLAIRAVMSEQDAYYLLAAVSKNPRPEAPCHLEAEQLSLTALEPALPDGHGFATVKLVFAPTAFMFHGRDVLYPSPQRIAYSLTKTYRDLFGTDLKQLADRAPTALELTHFAVRPVRVGIGEARAVPAYIGKAELAIYGNVETWLALLKLGQATGVGISRAIGFGKYKIKEVKKHA '

In [15]:
# Warnings suggested that there are frameshifted sequences. There are only 7, not to worry

err = 0
for index, gene in cas_voc.iterrows():
    if(len(gene["Seq"])%3 != 0):
        err += 1

err

7

In [18]:
output = "./cas_dataset_kira.tsv"
cas_voc.to_csv(output, sep='\t', index = False)

In [1]:
type(cas_voc)

NameError: name 'cas_voc' is not defined